<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Getting-Started" data-toc-modified-id="Getting-Started-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Getting Started</a></span></li><li><span><a href="#Autorization-Token" data-toc-modified-id="Autorization-Token-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Autorization Token</a></span></li><li><span><a href="#User-Play-History" data-toc-modified-id="User-Play-History-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>User Play History</a></span><ul class="toc-item"><li><span><a href="#Last-5-Songs-I-Listened-To" data-toc-modified-id="Last-5-Songs-I-Listened-To-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Last 5 Songs I Listened To</a></span></li><li><span><a href="#Last-Albums-I-Listened-To" data-toc-modified-id="Last-Albums-I-Listened-To-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Last Albums I Listened To</a></span></li></ul></li><li><span><a href="#Create-Markdown-of-Recent-Artists-for-Blog" data-toc-modified-id="Create-Markdown-of-Recent-Artists-for-Blog-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Markdown of Recent Artists for Blog</a></span></li><li><span><a href="#Final-Python-Script" data-toc-modified-id="Final-Python-Script-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Final Python Script</a></span></li></ul></div>

# Spotify Recently Played List!

I want to create a dynamicly updated list of artists I've been listening to based on my recent Spotify play history. This blog post is a work in progress as I work on this project :) 

<h2>Table of Contents<span class="tocSkip"></span></h2>
<div class="toc"><ul class="toc-item"><li><span><a href="#Getting-Started" data-toc-modified-id="Getting-Started-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Getting Started</a></span></li><li><span><a href="#Autorization-Token" data-toc-modified-id="Autorization-Token-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Autorization Token</a></span></li><li><span><a href="#User-Play-History" data-toc-modified-id="User-Play-History-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>User Play History</a></span><ul class="toc-item"><li><span><a href="#Last-5-Songs-I-Listened-To" data-toc-modified-id="Last-5-Songs-I-Listened-To-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Last 5 Songs I Listened To</a></span></li><li><span><a href="#Last-Albums-I-Listened-To" data-toc-modified-id="Last-Albums-I-Listened-To-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Last Albums I Listened To</a></span></li></ul></li><li><span><a href="#Create-Markdown-of-Recent-Artists-for-Blog" data-toc-modified-id="Create-Markdown-of-Recent-Artists-for-Blog-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Markdown of Recent Artists for Blog</a></span></li></ul></div>

In [1]:
from spotipy import util

from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import Spotify

import getpass

import pandas as pd

## Getting Started
1. Create a Spotify Developers Account 
2. Createe a new Application to get a client key and client secret - I named my application "Token Please" as I'm not using it for anything other than credentials
3. Installed the Spotipy package: `pip install spotipy`
4. Saved our Application credentials to our bash profile: `SPOTIPY_CLIENT_ID` and `SPOTIPY_CLIENT_SECRET`



## Autorization Token

Obtain an authorization token that we can use with the python and web API:
1. Get your userid by logging into Spotify, select the ... on your profile, click share, and then `copy spotify URI` 
2. Use the permissions we want as scope - ability to see what we've recently listened to
3. Use any valid URL as the redict URI

You will then be asked to log in to spotify (from a new webpage) and approve authorization. This will redirect you to a new webpage, copy this URL and paste in the text box below the `prompt_for_user_token` command which will then print your token.

In [ ]:
util.prompt_for_user_token(username="1238655357",
                           scope="user-read-recently-played",
                           redirect_uri='https://www.getpostman.com/oauth2/callback')

In [3]:
recent_play_token = getpass.getpass("Enter Your Token:")

Enter Your Token:········


## User Play History

In [4]:
spotify = Spotify(auth=recent_play_token)

### Last 5 Songs I Listened To

In [5]:
# get dictionary of songs from recent listening
last_songs_dict = spotify.current_user_recently_played(limit=5)['items']

# create dataframe of key information about each song
last_songs = pd.DataFrame()
for s in last_songs_dict:
    last_songs = last_songs.append(pd.DataFrame([[s['track']['name'], 
                                                  s['track']['artists'][0]['name'], 
                                                  s['track']['album']['name'],
                                                  s['played_at']]]))

last_songs = last_songs.reset_index(drop=True)
last_songs.columns = ["track_name", "artist_name", "album_title", "play_time"]

last_songs["play_time"] = pd.to_datetime(last_songs["play_time"])

last_songs

,track_name,artist_name,album_title,play_time
0,Let's Get Known,The Unicorns,Who Will Cut Our Hair When We're Gone? (Remast...,2020-05-13 00:14:31.853000+00:00
1,Child Star,The Unicorns,Who Will Cut Our Hair When We're Gone? (Remast...,2020-05-11 22:36:04.881000+00:00
2,The Clap,The Unicorns,Who Will Cut Our Hair When We're Gone? (Remast...,2020-05-11 22:30:42.982000+00:00
3,Jellybones,The Unicorns,Who Will Cut Our Hair When We're Gone? (Remast...,2020-05-11 22:29:16.240000+00:00
4,Sea Ghost,The Unicorns,Who Will Cut Our Hair When We're Gone? (Remast...,2020-05-11 22:26:32.334000+00:00


### Last Albums I Listened To
This is names of albums from recent songs rather than a list of albumns I listened fully to

In [6]:
last_songs_dict = spotify.current_user_recently_played()['items']

last_albumns = pd.DataFrame()
for s in last_songs_dict:
    
    last_albumns = last_albumns.append(pd.DataFrame([[s['track']['artists'][0]['name'], 
                                                      s["track"]["artists"][0]["uri"],
                                                      s['track']['album']['name']]]))

last_albumns.columns = ["artist_name", "artist_uri", "album_title"]

last_albumns = last_albumns.drop_duplicates()
last_albumns = last_albumns.reset_index(drop=True)

last_albumns

,artist_name,artist_uri,album_title
0,The Unicorns,spotify:artist:7L5HH5QtkDe7u2hJ1FUKFo,Who Will Cut Our Hair When We're Gone? (Remast...
1,Saves The Day,spotify:artist:5gWhlJBlLQGLOgYWO8lwCU,Stay What You Are
2,Patti Smith,spotify:artist:0vYkHhJ48Bs3jWcvZXvOrP,Horses (Legacy Edition)
3,Bob Dylan,spotify:artist:74ASZWbe4lXaubB36ztrGX,Highway 61 Revisited
4,Veruca Salt,spotify:artist:2QwJQuBekTA4qF7N7uLHDP,Eight Arms To Hold You
5,Jack's Mannequin,spotify:artist:42aeGx2I3uXINpGqC8L0LD,Everything In Transit
6,Gorillaz,spotify:artist:3AA28KZvwAUcZuOKwyblJQ,The Now Now
7,Radiohead,spotify:artist:4Z8W4fKeB5YxbusRsdQVPb,OK Computer
8,Foster The People,spotify:artist:7gP3bB2nilZXLfPHJhMdvc,Torches
9,The Strokes,spotify:artist:0epOFNiUfyON9EYx7Tpr6V,The New Abnormal


## Create Markdown of Recent Artists for Blog
Someday, we will want this dynamic, but for now here is the script to create the markdown for our recently listened to artists

In [7]:
last_songs_dict = spotify.current_user_recently_played()['items']

artists = pd.DataFrame()

# Look at each track
for s in last_songs_dict:
    # Look at each artist that contributed to the track
    for a in s['track']['artists']:
        
        # save artist name, spotify reference uri, and link to their spotify page
        artists = artists.append(pd.DataFrame([[
            a["name"], a["uri"], a["external_urls"]["spotify"]
        ]]))

artists.columns = ["artist_name", "artist_uri", "artist_page"]

artist_counts = artists.groupby(["artist_name", "artist_uri", "artist_page"]).size().reset_index()

artist_counts.columns = ["artist_name", "artist_uri", "artist_page", "song_count"]

for i, r in artist_counts.sort_values("song_count", ascending=False).iterrows():
    print("* [" + r["artist_name"] + "](" + r["artist_page"] +")")

* [Patti Smith](https://open.spotify.com/artist/0vYkHhJ48Bs3jWcvZXvOrP)
* [Saves The Day](https://open.spotify.com/artist/5gWhlJBlLQGLOgYWO8lwCU)
* [The Unicorns](https://open.spotify.com/artist/7L5HH5QtkDe7u2hJ1FUKFo)
* [Bob Dylan](https://open.spotify.com/artist/74ASZWbe4lXaubB36ztrGX)
* [Jack's Mannequin](https://open.spotify.com/artist/42aeGx2I3uXINpGqC8L0LD)
* [Gorillaz](https://open.spotify.com/artist/3AA28KZvwAUcZuOKwyblJQ)
* [Blur](https://open.spotify.com/artist/7MhMgCo0Bl0Kukl93PZbYS)
* [Fatoumata Diawara](https://open.spotify.com/artist/4G5ZJny3HvX6Il7eHVfnNC)
* [Foster The People](https://open.spotify.com/artist/7gP3bB2nilZXLfPHJhMdvc)
* [Interpol](https://open.spotify.com/artist/3WaJSfKnzc65VDgmj2zU8B)
* [Jamie Principle](https://open.spotify.com/artist/5obQFNrkFoWB51hm1JTHMw)
* [Radiohead](https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb)
* [Snoop Dogg](https://open.spotify.com/artist/7hJcb9fa4alzcOq3EaNPoG)
* [The Strokes](https://open.spotify.com/artist/0epOFNiUfy

## Final Python Script

`./<FILE_NAME.py> $SPOTIFY_TOKEN`

```python
#!/usr/local/bin/python3
# Michelle Tanco - michelle.tanco@gmail.com
# Create Markdown of Spotify Play History

from spotipy import Spotify
import pandas as pd
import sys

if len(sys.argv) < 2:
    print("Please include your Spotify Token!")
    print("./<file_name.py> $SPOTIFY_TOKEN")
    sys.exit

spotify = Spotify(auth=str(sys.argv[1]))

last_songs_dict = spotify.current_user_recently_played()['items']

artists = pd.DataFrame()

# Look at each track
for s in last_songs_dict:
    # Look at each artist that contributed to the track
    for a in s['track']['artists']:
        # save artist name, spotify reference uri, and link to their spotify page
        artists = artists.append(pd.DataFrame([[
            a["name"], a["uri"], a["external_urls"]["spotify"]
        ]]))

artists.columns = ["artist_name", "artist_uri", "artist_page"]

artist_counts = artists.groupby(["artist_name", "artist_uri", "artist_page"]).size().reset_index()

artist_counts.columns = ["artist_name", "artist_uri", "artist_page", "song_count"]

for i, r in artist_counts.sort_values("song_count", ascending=False).iterrows():
    print("* [" + r["artist_name"] + "](" + r["artist_page"] + ")")

```